In [1]:
from google import google
import gnp
import codecs
import json
from boto.s3.connection import S3Connection
from boto.s3.key import Key
import boto
import sys, os
import pandas as pd
from TwitterSearch import *
import sunlight
from sunlight import congress
from bs4 import BeautifulSoup
import urllib2
import itertools
import datetime
import configparser
import time
from pattern.en import parsetree
from pattern.en import mood

In [2]:
action_categories = ["donation","give","donate"
                     "protest","rally","demonstration","demonstrate",
                     "meetup","huddle","congregate","join",
                     "boycott",
                     "call","email","reach out","senator","representative"
                     "vote",
                     "sign","petition",
                     "town hall","open office","town meeting","townhall",
                     "school board meeting"]

issues = ["civil rights",
"women's rights","planned parenthood",
"black lives matter","african american rights","black rights",
"immigration","immigrants","undocumented","border wall","refugees","refugee rights",
"gun control",
"public education",
"LGBTQ rights","LGBTQ","LGBT",
"voting rights","redistricting","voter id laws","voting access","voter access"
         ]

legislators = pd.read_csv('data/static_data/legislators.csv')

mask = (legislators.in_office == 1)

legislators = legislators.loc[mask]

legislators['title_name'] = legislators['full_title'] + ' ' + legislators['firstname'] + ' ' + legislators['lastname']

print legislators.columns
print
print list(legislators['title_name'])[:10]

leg_names = list(legislators['title_name'])
leg_states = list(legislators['state'])

keyword_searches = []

for combination in itertools.product(action_categories, issues):
  keyword_searches.append(list(combination))
  
print keyword_searches[:20]

Index([u'title', u'full_title', u'firstname', u'middlename', u'lastname',
       u'name_suffix', u'nickname', u'party', u'state', u'district',
       u'in_office', u'gender', u'phone', u'fax', u'website', u'webform',
       u'congress_office', u'bioguide_id', u'votesmart_id', u'fec_id',
       u'govtrack_id', u'crp_id', u'twitter_id', u'congresspedia_url',
       u'youtube_url', u'facebook_id', u'official_rss', u'senate_class',
       u'birthdate', u'oc_email', u'title_name'],
      dtype='object')

['Representative Robert Aderholt', 'Senator Lamar Alexander', 'Representative Justin Amash', 'Representative Mark Amodei', 'Representative Alma Adams', 'Representative Pete Aguilar', 'Representative Rick Allen', 'Representative Ralph Abraham', 'Representative Jodey Arrington', 'Representative Joe Barton']
[['donation', 'civil rights'], ['donation', "women's rights"], ['donation', 'planned parenthood'], ['donation', 'black lives matter'], ['donation', 'african american rights'], ['donation',

In [5]:
i = 0
num_page = 3
for search in keyword_searches:
  search_results = google.search(' '.join(search), num_page)
  with open('data4.txt', 'a') as outfile:
    for result in search_results:
      new_record = {
        'source': 'Google',
        'text': result.description,
        'url': result.link,
        'keywords':search,
#         'created_date': ,
        'search_date': datetime.datetime.today().strftime("%m/%d/%Y")
      }
      json.dump(new_record, outfile)
      outfile.write('\n')
      i += 1
      if i % 100 ==0:
        print( result.description  )
        print result.link
        print search



Aug 10, 2016 ... The Weeknd recently donated $250,000 to the Black Lives Matter Network, two 
sources familiar with the situation told The FADER on ...
http://www.thefader.com/2016/08/10/the-weeknd-donate-black-lives-matter
['donation', 'black lives matter']
Dec 20, 2016 ... Council Speaker Melissa Mark-Viverito, center, joins lawmakers in blasting 
fraudulent immigration services in front of the City Hall steps.
http://observer.com/2016/12/new-york-electors-donate-salaries-to-pro-immigrant-group-in-defiance-of-trump/
['donation', 'immigration']
Dec 25, 2015 ... There are about 4.4 million Syrian refugees outside Syria, according to ... Several 
tools below provide guidance on where to donate, as well as ...
https://www.nytimes.com/2015/12/26/your-money/how-to-help-in-a-global-refugee-crisis.html
['donation', 'refugees']
Investigators at Claremont Graduate University, in collaboration with the Donor 
Network of Arizona, will replicate and extend a letter-based direct mail campaign
 ...

KeyboardInterrupt: 

In [7]:
config = configparser.ConfigParser()
config.read('ross.ini')

consumer_key = config['twitter.api']['consumer_key']
consumer_secret = config['twitter.api']['consumer_secret']
access_token = config['twitter.api']['access_token']
access_token_secret = config['twitter.api']['access_token_secret']

AWS_ACCESS_KEY_ID = config['aws.creds']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = config['aws.creds']['AWS_SECRET_ACCESS_KEY']

/home/rkatz/.local/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  from ipykernel import kernelapp as app


In [8]:
bucket_name = 'mids-capstone-rzst'
conn = boto.connect_s3(AWS_ACCESS_KEY_ID,AWS_SECRET_ACCESS_KEY)


# bucket = conn.create_bucket(bucket_name,
#     location=boto.s3.connection.Location.DEFAULT)

testfile = "data4.txt"
print 'Uploading %s to Amazon S3 bucket %s' % \
   (testfile, bucket_name)

def percent_cb(complete, total):
    sys.stdout.write('.')
    sys.stdout.flush()


# k = Key(bucket_name)
# k.key = 'data.txt'
# k.set_contents_from_filename(testfile,
#     cb=percent_cb, num_cb=10)


# s3_connection = boto.connect_s3()
bucket = conn.get_bucket('mids-capstone-rzst')
key = boto.s3.key.Key(bucket, 'data4.txt')
key.set_contents_from_filename('data4.txt')


Uploading data4.txt to Amazon S3 bucket mids-capstone-rzst


480367

In [ ]:
reload(sys)  
sys.setdefaultencoding('UTF8')


zip_geo = pd.read_csv('/Users/rkatz/Documents/Zip_lat_long.csv')

zips = []

for index, row in zip_geo.iterrows():
    new_record = {
        'zip': row['ZIP'],
        'lat': row['LAT'],
        'lng': row['LNG']
    }
    zips.append(new_record)